# **Lab 3: Exploratory Data Analysis and Data Cleaning**

Today, we are going to be conducting **exploratory data analysis (EDA)** on a dataset of trending YouTube videos. We will be creating key data visualizations, summarizing our findings with descriptive statistics, and trying to identify which groups or topics of videos perform best and worst!

First, let's import our key libraries. We've already seen `pandas` and `numpy` extensively, but today we will be working with `matplotlib` and `seaborn` for the first time. These libraries allow us to create data visualizations.

In [ ]:
# JUST RUN THIS CELL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

We are also giving you a crude subtopic modeling function, which leverages a **transformer** model that will group YouTube videos into subtopics based on title and channel name.

Note that this cell may take around 30 seconds to run so do not fret!

In [ ]:
# JUST RUN THIS CELL

from transformers import pipeline

# Load zero-shot classification model
classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-small")

subtopic_labels = ["trump", "clinton", "nba", "roy moore", "net neutrality", "ajit pai", "elections", "republicans", "democrats", "special needs",
                   "movie trailer", "tina nguyen", "lawsuits", "activism", "book review", "supreme court", "jimmy kimmel", "youtube drama", "celebrity news"]

def get_subtopics(title: str, subtopic_labels: list) -> str:
    result = classifier(title, subtopic_labels)
    return result["labels"][0]  # Return highest confidence label

get_subtopics("Insane Three-Point Shot Made by Stephen Curry! | Lakers vs. Warriors Basketball Highlights NBA", subtopic_labels)

## Task 1: Load in the Data

The dataset will be at the following URL below:

In [ ]:
# JUST RUN THIS CELL
URL = "https://raw.githubusercontent.com/ArnavG/saas_workshops/refs/heads/main/USvideos.csv.zip"

You can use `pandas.read_csv()` to open the zip file in the URL, just like you always have! (Under the hood, the function is inferring that the file type is a zip file and passing in arguments to handle this file type. In other cases, we might not get so lucky.)

In [ ]:
df = ...
df

## Task 2: Check which Columns have Null or Missing Values

- **Hint**: use `pandas.DataFrame.isna()` and a certain aggregation function
  - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html

In [ ]:
...

## Task 3 - DateTime Conversion

We have an existing DateTime column called `publish_time`, which tells us when the video was uploaded to YouTube, but doesn't tell us when the video actually started trending. For that, we need to look at the `trending_date` column.

To convert each date in `trending_date` to a DateTime object, we can use the following code syntax:

In [ ]:
# JUST RUN THIS CELL

date_str = "17.14.11" # Sample date
date_obj = pd.to_datetime(date_str, format="%y.%d.%m") # Convert to datetime (YY.DD.MM)
print(date_obj)

Now, convert all the dates in `trending_date` into DateTime format. You may either use `pandas.Series.apply()` or take advantage of vectorized operations.
- https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html

In [ ]:
df["trending_date"] = ...
df.head()

Now, we'll create a new column called `trending_date_year_month` that takes the DateTime object in the `trending_date` column and converts each one into a `YYYY-MM` format, like so:

In [ ]:
# JUST RUN THIS CELL
df["trending_date_year_month"] = df["trending_date"].dt.strftime('%Y-%m')
df.head()

## Task 4 - Aggregating the Data

Sometimes, the same YouTube video will appear on the trending page multiple times over the course of many days. It may be off the trending page at one point, then find its way back on, then off, then on again, etc.

This complicates things for our analysis because the same video will appear with different view, like, dislike, and comment counts, multiple times in the dataset. However, we can't simply take the sum of these engagement metrics, because they represent the total engagement numbers at that moment in time, not the changes since the last time the video was trending.

For example, take the video titled `"Judge Roy Moore Campaign Statement"`

In [ ]:
# JUST RUN THIS CELL
df[df["title"] == "Judge Roy Moore Campaign Statement"]

It first hit trending with on December 14, 2017, with 51,139 views, 441 likes, and 4693 dislikes, but over the course of one week, it accumulated more and more views, likes, and dislikes (especially dislikes). Comments appear to have been disabled. (You should go look up who Roy Moore is if you are unfamiliar to find out why this might be.)

To get the number of views, likes, dislikes, and comments for each video, we need to group by video title, but instead of aggregating by **summing** the engagement metrics, we need to use another aggregation function. What should this function be?

1. Group the `df` dataframe by all **categorical features** and find the final number of views,likes, dislikes, and comments that each video had.
2. Do not omit any columns.
3. Save this grouped dataframe as `trended_videos`. Chain the `reset_index()` function to the end of your code as well.

In [ ]:
# STEP 1 - Create a categorical features list
# You can either hard-code this, or you can get creative and use .dtypes and boolean filtering to get them. I would recommend the latter as a good brain teaser
categorical_columns = ...
categorical_columns

In [ ]:
# STEP 2 - Create the trended_videos dataframe with the correct groupby() syntax
trended_videos = ...
trended_videos

In [ ]:
# JUST RUN THIS CELL
trended_videos[trended_videos["title"] == "Judge Roy Moore Campaign Statement"]

Does this match what you expected? If not, double check your grouping and aggregation functions.

We will be working with the `trended_videos` dataframe for subequent tasks as well.

## Task 5 - The YouTube Channels that Trended the Most


### 5.1
Get the **number of times** each YouTube video in the dataset appeared on the trending page. Sort this dataframe by the number of trending page appearances from greatest to least. Store the result in a dataframe called `trending_counts`.

- **Hint**: use `pandas.DataFrame.groupby()`
  - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
- **Hint**: think about what aggregation function you will need to use to get the *number* of times each title appeared in the trending dataset (if you are stuck, ask for help!)
- **Hint**: use `pandas.DataFrame.sort_values()`
  - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

In [ ]:
trending_counts = ...
trending_counts

### 5.2
Create a histogram of the number of times each channel appeared on the trending page. You may either use:

- `matplotlib.pyplot.hist()`
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html

or

- `seaborn.histplot()`
  - https://seaborn.pydata.org/generated/seaborn.histplot.html

to create your histogram. Be mindful of the parameters you pass in. Feel free to search up other plotting documentation or examples. I would also recommend sending the parameter `kde=True`

In [ ]:
...

### 5.3

Let's generate some summary statistics for this trending counts distribution. Print out the following information:

- The **minimum** of the distribution (hint: `numpy.min()`)
- The **1st percentile** of the distribution (hint: `numpy.quantile()`)
  - https://numpy.org/doc/stable/reference/generated/numpy.quantile.html
- The **5th percentile** of the distribution
- The **10th percentile** of the distribution
- The **25th percentile** of the distribution
- The **median** of the distribution (hint: `numpy.median()`)
- The **mean** of the distribution (hint: `numpy.mean()`)
- The **75th percentile** of the distribution
- The **90th percentile** of the distribution
- The **95th percentile** of the distribution
- The **99th percentile** of the distribution
- The **maximum** of the distribution (hint: `numpy.max()`)

In [ ]:
# Generate summary statistics
print("Min:", ...)
print("1st Percentile:", ...)
print("5th Percentile:", ...)
print("10th Percentile:", ...)
print("25th Percentile:", ...)
print("Median:", ...)
print("Mean:", ...)
print("75th Percentile:", ...)
print("90th Percentile:", ...)
print("95th Percentile:", ...)
print("99th Percentile:", ...)
print("Max:", ...)

### 5.4

Summarize your findings below. Comment on the shape, center, and spread of the distribution of trending video counts, citing relevant summary statistics and interpreting the data in context.

(YOUR ANSWER GOES HERE.)

## Task 6 - Trends and Engagement Over Time

### 6.1

While not a perfect measure of platform activity, let's create a column in `trended_videos` called `"Engagement"` that equals the sum total of `views`,	`likes`, `dislikes`, and `comment_count`.

In [ ]:
trended_videos["Engagement"] = ...
trended_videos.head()

### 6.2

Now, create a dataframe that calculates **total monthly engagement** for each `trending_date_year_month` date that appears in the dataframe. You will need to use `pandas.DataFrame.groupby()`. Save the grouped dataframe as `monthly_engagement` with a date column and a total engagement column.
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

- **HINT**: you may need to use `pandas.DataFrame.reset_index()`
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

In [ ]:
monthly_engagement = ...
monthly_engagement

### 6.3

Now, using the `monthly_engagement` dataframe, create a time series line plot that plots total engagement for each month. You may either use:
- `matplotlib.pyplot.plot()`
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

or

- `seaborn.lineplot()`
  - https://seaborn.pydata.org/generated/seaborn.lineplot.html

In [ ]:
...

### 6.4 Comment on your findings. What trend do you observe? Why might this be the case?

(YOUR ANSWER GOES HERE)

### 6.5 Decomposing Engagement

It can be helpful to know which components of engagement are actually driving the trend we observe. Redo the exercise from 4.3, but instead of aggregating just the `Engagement` column, aggregate the `views`, `likes`, `dislikes`, and `comment_count` columns and save the grouped and aggregated data into a dataframe called `monthly_components`.
- **HINT**: you will need to use `pandas.DataFrame.reset_index()`
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

In [ ]:
monthly_components = ...
monthly_components

### 6.6 Four Different Lineplots

Now, we are going to plot all four engagement variables on the same set of axes. Our first step is to generate a "massaged" version of our dataframe using `pandas.melt()`. This will create a dataframe that has each value associated with its respective engagement variable (likes, dislikes, comment count, or views) and the corresponding video.

![pd.melt](https://pandas.pydata.org/docs/_images/reshaping_melt.png)

Use the following documentation:
- https://pandas.pydata.org/docs/reference/api/pandas.melt.html
- **HINT**: you will need to pass `"trending_date_year_month"` as the `id_vars` parameter in the function

In [ ]:
monthly_melted = ...
monthly_melted

Now, use either matplotlib or seaborn to generate a plot of 4 different time series for each engagement metric. Some helpful documentation below:
- Matplotlib: https://www.geeksforgeeks.org/plot-multiple-lines-in-matplotlib/
- Seaborn: https://stackoverflow.com/questions/52308749/how-do-i-create-a-multiline-plot-using-seaborn

In [ ]:
...

### 6.7 If you worked as a data scientist at YouTube, which metric would you be trying to optimize to increase engagement and why?

(YOUR ANSWER GOES HERE.)

## Task 7 - Correlated Metrics

### 7.1

Let's start by calculating the correlation coefficient between each pair of metrics: `views`, `likes`, `dislikes`, and `comment_count`.

Rather than iterating over each one and manually calculating the correlation coefficient, we can make use of the function `pandas.DataFrame.corr()`.
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html

This will generate a **symmetric matrix** of the pair-wise correlation of each metric in our dataframe.

As a first step, let's create a dataframe called `metrics` that only contains the columns `views`, `likes`, `dislikes`, and `comment_count` from `trended_videos`. Then, use the resulting dataframe and the `corr()` function above to generate a correlation matrix called `corr_matrix`.

In [ ]:
metrics = ...
metrics

In [ ]:
corr_matrix = ...
corr_matrix

### 7.2 Correlation Heatmap

Now, use the `seaborn` library to create a correlation **heatmap**. Documentation for the `seaborn.heatmap()` plotting function is below:
- https://seaborn.pydata.org/generated/seaborn.heatmap.html

**HINT**: if you want to display the correlation numbers on the heatmap, set the `annot` parameter accordingly.

**BONUS**: One of Arnav's biggest pet peeves is poorly-colored correlation heatmaps, and it is his humble opinion that the default colors are terrible. If you feel the same way, Arnav would encourage you to play around with the color palette and make a more presentable heatmap. (You can change the `cmap` parameter.)

In [ ]:
...

### 7.3

Interpret the correlation heatmap. Which variables are the most highly correlated? The least? Why do you think this is? If two variables have a correlation close to 1, what does that imply about their linear relationship?

(YOUR ANSWER GOES HERE.)

### 7.4 - Scatterplots

Create a scatterplot of the relationship between `likes` and `comment_count`. You may use either:
- `matplotlib.pyplot.scatter()`
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

or

- `seaborn.scatterplot()`
  - https://seaborn.pydata.org/generated/seaborn.scatterplot.html

**Make sure you add a title and axis labels!** To do that, you can use:
- `matplotlib.pyplot.title`
  - https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.title.html
- `matplotlib.pyplot.xlabel` and `matplotlib.pyplot.ylabel`
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlabel.html
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ylabel.html

In [ ]:
...

Now, create a scatterplot with labeled axes and a title for the relationship between `likes` and `dislikes`.

In [ ]:
...

### 7.5

Are these plots what you expected given their correlation? Explain your reasoning as to why the plots might look the way they do in spite of the correlation coefficient. What does this tell you about the correlation coefficient (what types of associations can it measure)?

(YOUR ANSWER GOES HERE.)

## Task 8 - Disliked Videos (Topic Modeling)

Let's investigate the most **disliked** YouTube videos that trended – videos that perhaps got popular for all the wrong reasons. Additionally, we'll make use of the two functions we gave you at the beginning of the lab (`get_topics` and `get_subtopics`) and do **topic modeling** to find out which "themes" or "groups" of topics were the most disliked. (This is a common task in natural language processing [NLP], but we unfortunately won't be able to go over it today...story for another day! Probably in DM.)

### 8.1 Selecting Specific Columns

Let's create a new dataframe called `likes_and_dislikes` that only has the following columns from `trended_videos`:
- `trending_date`
- `title`
- `channel_title`
- `category_id`
- `likes`
- `dislikes`

In [ ]:
likes_and_dislikes = ...
likes_and_dislikes

### 8.2 String Column Concatenation

Create a column in `likes_and_dislikes` called `"title_and_channel"`. This should be the result of concatenating the `"title"` and `"channel_title"` columns with a space (`" "`) in between. Your answer should make use of pandas/numpy vectorization abilities for Series/arrays containing strings!

In [ ]:
likes_and_dislikes["title_and_channel"] = ...
likes_and_dislikes

### 8.3 Computing the Likes to Dislike Ratio

Add a column called `like_dislike_ratio` to the `likes_and_dislikes` dataframe that contains entries for the like to dislike ratio for each video. Then, sort the dataframe by like to dislike ratio, starting from the most disliked videos to the least disliked videos (by ratio).

In [ ]:
likes_and_dislikes["like_dislike_ratio"] = ...
likes_and_dislikes = ...
likes_and_dislikes.head(30)

### 8.4 Most Disliked Videos

Now, create a dataframe called `most_disliked` that only contains videos in `likes_and_dislikes` that have more dislikes than likes. (What can you filter `like_dislike_ratio` by to achieve this?)

In [ ]:
most_disliked = ...
most_disliked

### 8.5 Most Disliked Topics with Web Scraping

Now, we can finally make use of the topic modeling functions we defined earlier to see which "themes" or "topics" of videos get the most dislikes.

Before we do that, though, YouTube actually has its own designated video category IDs. You can find them on the GitHub raw page below:
- https://gist.githubusercontent.com/dgp/1b24bf2961521bd75d6c/raw/9abca29c436218972e6e28de7c03060845ed85d9/youtube%2520api%2520video%2520category%2520id%2520list

Instead of manually hard-coding a list of these categories, let's get some practice automating this process using **web scraping**! For this task, we will use the `BeautifulSoup` library. We will import this library along with the `requests` module, which allows us to make HTTP requests to a specific website using Python.

In [ ]:
# JUST RUN THIS CELL - Import Web Scraping Tools
import requests
from bs4 import BeautifulSoup

Next, fill in the `url` variable with the link to the webpage we are trying to scrape. After you do this, notice that:

1. We first make a request to the specified URL using `requests.get(url)`. In more technical terms, what this does is retrieve data from a server in the form of the webpage. When a URL is entered into a web browser, the browser sends a GET request to the server hosting the website, and the server responds by sending the requested data back.
2. We then use `BeautifulSoup` to get the `.text` attribute from the requested webpage using `html.parser`. This is essentially what does the scraping task for us and loads in all the text from the requested webpage.

In [ ]:
url = ... # YOUR CODE GOES HERE (copy the GitHub URL here)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

Now, we can use the `find_all()` function from `BeautifulSoup` to generate all elements on the webpage for us.

In [ ]:
# JUST RUN THIS CELL
texts = soup.find_all(string=True)
texts

**FINALLY, YOUR TASK:**

The `texts` variable is a one-element list containing all the string text from the category IDs GitHub raw webpage.

Each category ID in the list is separated by the newline separator, `'\n'`. We are also interested in only the first 32 category IDs presented in the list. (The rest are redundant.)

Your task is to use Python string parsing functions to create a pandas dataframe called `df_categories` that contains two columns:
1. `category_id`, an **integer** column of all category IDs in the list
2. `category_name`, which is the corresponding name for each category ID in the list.

This is an open-ended question, but it can also be challenging! Here are some Python tricks and functions our solution used:

- used the `string.split()` function twice
- used list slicing to get the first 32 category IDs
- created two lists/arrays to store each category ID and name and iterated through all of the given categories
- used `np.array([]).astype(int)` to convert the list of category IDs into an integer array rather than a string array
- created a new pandas Dataframe with two columns called `category_id` and `category_name`

In [ ]:
...

df_categories = pd.DataFrame({...})
df_categories

Next, merge the `df_categories` dataframe categories with the categories in `most_disliked`. No rows should get ddropped from `most_disliked`. Each one should have a corresponding category ID from `df_categories`.

You can call the resulting dataframe `most_disliked_categories`.

A reminder on how the pandas merge syntax works:
- https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge

In [ ]:
most_disliked_categories = ...
most_disliked_categories

Now, let's generate two **bar plots**:

1. A bar plot of the most disliked `category_name`s by count
2. A bar plot of each `category_name`'s average like to dislike ratio

You know the drill by now: use grouping and aggregation to create the intermediate dataframes, then use a plotting library to generate bar plots. You may either use:

- `matplotlib.pyplot.bar()`
  - https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html

or

- `seaborn.barplot()`
  - https://seaborn.pydata.org/generated/seaborn.barplot.html


If your x-axis labels are overlapping, I would recommend adding the following code above your bar plot:

`plt.figure(figsize=(20, 10))`

In [ ]:
most_disliked_counts = ...
most_disliked_counts

In [ ]:
...

In [ ]:
most_disliked_avg = ...
most_disliked_avg

In [ ]:
...

### 8.6 Generating Subtopics with a Language Model (NLP)

If you look at these official YouTube categories, they may seem a little unspecific. For example, "News & Politics" gets the most dislikes, and "People & Blogs" has the lowest like to dislike ratio. However, "People & Blogs" also includes political videos like Roy Moore's concession speech. It would be better if we had a way to get more specific topics based on our data!

Let's use our `get_subtopics()` function defined at the beginning of the lab. This function makes use of the small language model DeBERTa to assign subtopic labels to each video in our dataset based on the title of the YouTube video and the channel name (`title_and_channel`).

As a reminder, the function syntax is below:

In [ ]:
# JUST RUN THIS CELL
get_subtopics("Insane Three-Point Shot Made by Stephen Curry! | Lakers vs. Warriors Basketball Highlights NBA", subtopic_labels)

**Now, for the task.**

Using `pandas.Series.apply()`, call the `get_subtopics()` function on the `title_and_channel` column in the `most_disliked_categories` dataframe. Save the resulting Series as a new column in `most_disliked_categories` called `subtopic`.
- **Hint**: Documentation for `pandas.Series.apply()` is below. Rather than pass in the function name directly, you should pass in a `lambda` function that calls `subtopic_classifier` and pass in the `subtopic_labels` you generated. Syntax examples below:

  - https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html
  - https://stackoverflow.com/questions/19914937/applying-function-with-multiple-arguments-to-create-a-new-pandas-column

- **NOTE**: This may take around 4 to 5 minutes to run. Don't worry! That's normal for more complicated ML tasks. Go take a stretch break while it runs :)

In [ ]:
most_disliked_categories["subtopic"] = ...
most_disliked_categories

### 8.7 Do you agree with the subtopic labels? Which ones seem correct and which ones seem off? Feel free to randomly sample from the dataframe to make your judgment.

(YOUR ANSWER GOES HERE.)

### 8.8 Subtopic Bar Plots

Just like you did for the category names, generate two bar plots for the subtopics:
1. One bar plot getting the count of each subtopic in the `most_disliked_categories` dataframe
2. One bar plot getting the average like to dislike ratio of each subtopic in the `most_disliked_categories` dataframe.

Feel free to copy-paste and modify your code from above for this question!

In [ ]:
most_disliked_subtopics_counts = ...
most_disliked_subtopics_counts

In [ ]:
...

In [ ]:
most_disliked_subtopics_avg = ...
most_disliked_subtopics_avg

In [ ]:
...

# Congratulations! You have finished lab 3 🎉

Make sure you submit your lab notebook to Gradescope!